In [ ]:
#1페이지만 실행(test)

import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

# 크롬 드라이버 경로 설정
CHROME_DRIVER_PATH = r'C:\thesis\chromedriver.exe'

# Selenium WebDriver 설정
options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # 디버그용으로 브라우저 표시
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# ChromeDriver 서비스 설정
service = Service(executable_path=CHROME_DRIVER_PATH)
driver = webdriver.Chrome(service=service, options=options)

# 사이트 접속
url = 'https://ekalba.lt/bendrines-lietuviu-kalbos-zodynas/'
driver.get(url)
print("💡 페이지가 완전히 로딩되었는지 확인하고 'yes'를 입력하세요:")
input("✅ 준비되면 'yes'를 입력하고 Enter를 누르세요: ")

# CSV 파일 설정
OUTPUT_CSV = 'lithuanian_words_1page.csv'
with open(OUTPUT_CSV, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Word', 'Part of Speech', 'Dark Text', 'Description', 'Additional Info', 'Update Date'])

    print(f'🔄 Processing page 1 of 1...')
    try:
        # 페이지 소스 파싱
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        words = soup.select('a.words_list__links span.ng-binding.ng-scope')  # 단어 목록 영역만 선택

        # 단어 클릭 후 데이터 수집
        for word in words:
            word_text = word.get_text().strip()
            try:
                print(f"🔍 Found word element: {word_text}")

                # 클릭 전 URL 기록
                old_url = driver.current_url

                # JavaScript를 이용하여 직접 요소 찾기 및 클릭
                word_element = driver.execute_script("""
                    let elements = document.querySelectorAll("a.words_list__links span.ng-binding.ng-scope");
                    for (let el of elements) {
                        if (el.innerText.trim() === arguments[0]) {
                            el.click();
                            return el;
                        }
                    }
                    return null;
                """, word_text)

                if not word_element:
                    print(f"❌ Word element not found via JavaScript: {word_text}")
                    continue

                print(f"✅ 강제 클릭 성공: {word_text}")

                # URL이 변경될 때까지 대기
                WebDriverWait(driver, 20).until(
                    lambda driver: driver.current_url != old_url
                )
                print(f"🔄 URL 변경 감지: {driver.current_url}")

                # 추가 대기: AngularJS의 동적 로딩 대기
                time.sleep(2)

                # JavaScript로 동적으로 렌더링된 데이터를 가져옴
                word_details = driver.execute_script("""
                    const data = {};
                    const partOfSpeechElement = document.querySelector("span.bzpetitas");  // 수정된 부분
                    const darkTextElement = document.querySelector("span.bzpusjuodis");
                    const descriptionElements = document.querySelectorAll("span.bzkursyvas");
                    const additionalInfoElements = document.querySelectorAll("span.bzpaprastas");
                    const updateDateElement = document.querySelector("p.bz-update-date");

                    data.partOfSpeech = partOfSpeechElement ? partOfSpeechElement.innerText : "N/A";
                    data.darkText = darkTextElement ? darkTextElement.innerText : "N/A";
                    data.descriptions = [];
                    data.additionalInfos = [];
                    data.updateDate = updateDateElement ? updateDateElement.innerText : "N/A";

                    descriptionElements.forEach((el) => {
                        data.descriptions.push(el.innerText);
                    });
                    additionalInfoElements.forEach((el) => {
                        data.additionalInfos.push(el.innerText);
                    });

                    return data;
                """)

                part_of_speech = word_details['partOfSpeech']
                dark_text = word_details['darkText']
                description = " | ".join(word_details['descriptions'])
                additional_info = " | ".join(word_details['additionalInfos'])
                update_date = word_details['updateDate']

                # CSV 파일에 저장
                writer.writerow([word_text, part_of_speech, dark_text, description, additional_info, update_date])
                print(f'✅ Successfully saved: {word_text}')

                # 상세 페이지에서 뒤로가기
                driver.back()
                time.sleep(2)

            except Exception as e:
                print(f'❌ Failed to process word: {word_text} - {str(e)}')
                continue

    except Exception as e:
        print(f"❌ 페이지 처리 실패: {str(e)}")

# 드라이버 종료
driver.quit()
print('🎉 Crawling completed and saved to', OUTPUT_CSV)


In [ ]:
#전체 단어목록 추출 코드

import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

# 크롬 드라이버 경로 설정
CHROME_DRIVER_PATH = r'C:\thesis\chromedriver.exe'

# Selenium WebDriver 설정
options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # 디버그용으로 브라우저 표시
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# ChromeDriver 서비스 설정
service = Service(executable_path=CHROME_DRIVER_PATH)
driver = webdriver.Chrome(service=service, options=options)

# 사이트 접속
url = 'https://ekalba.lt/bendrines-lietuviu-kalbos-zodynas/'
driver.get(url)
print("💡 페이지가 완전히 로딩되었는지 확인하고 'yes'를 입력하세요:")
input("✅ 준비되면 'yes'를 입력하고 Enter를 누르세요: ")

# 페이지 수 계산 (1505 페이지)
total_pages = 1505

# CSV 파일 설정
OUTPUT_CSV = 'lithuanian_words_all_pages.csv'
with open(OUTPUT_CSV, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Word', 'Part of Speech', 'Dark Text', 'Description', 'Additional Info', 'Update Date'])

    for page in range(1, total_pages + 1):
        print(f'🔄 Processing page {page} of {total_pages}...')
        try:
            # 페이지 소스 파싱
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            words = soup.select('a.words_list__links span.ng-binding.ng-scope')  # 단어 목록 영역만 선택

            # 단어 클릭 후 데이터 수집
            for word in words:
                word_text = word.get_text().strip()
                try:
                    print(f"🔍 Found word element: {word_text}")

                    # 클릭 전 URL 기록
                    old_url = driver.current_url

                    # JavaScript를 이용하여 직접 요소 찾기 및 클릭
                    word_element = driver.execute_script("""
                        let elements = document.querySelectorAll("a.words_list__links span.ng-binding.ng-scope");
                        for (let el of elements) {
                            if (el.innerText.trim() === arguments[0]) {
                                el.click();
                                return el;
                            }
                        }
                        return null;
                    """, word_text)

                    if not word_element:
                        print(f"❌ Word element not found via JavaScript: {word_text}")
                        continue

                    print(f"✅ 강제 클릭 성공: {word_text}")

                    # URL이 변경될 때까지 대기
                    WebDriverWait(driver, 20).until(
                        lambda driver: driver.current_url != old_url
                    )
                    print(f"🔄 URL 변경 감지: {driver.current_url}")

                    # 추가 대기: AngularJS의 동적 로딩 대기
                    time.sleep(2)

                    # JavaScript로 동적으로 렌더링된 데이터를 가져옴
                    word_details = driver.execute_script("""
                        const data = {};
                        const partOfSpeechElement = document.querySelector("span.bzpetitas");
                        const darkTextElement = document.querySelector("span.bzpusjuodis");
                        const descriptionElements = document.querySelectorAll("span.bzkursyvas");
                        const additionalInfoElements = document.querySelectorAll("span.bzpaprastas");
                        const updateDateElement = document.querySelector("p.bz-update-date");

                        data.partOfSpeech = partOfSpeechElement ? partOfSpeechElement.innerText : "N/A";
                        data.darkText = darkTextElement ? darkTextElement.innerText : "N/A";
                        data.descriptions = [];
                        data.additionalInfos = [];
                        data.updateDate = updateDateElement ? updateDateElement.innerText : "N/A";

                        descriptionElements.forEach((el) => {
                            data.descriptions.push(el.innerText);
                        });
                        additionalInfoElements.forEach((el) => {
                            data.additionalInfos.push(el.innerText);
                        });

                        return data;
                    """)

                    part_of_speech = word_details['partOfSpeech']
                    dark_text = word_details['darkText']
                    description = " | ".join(word_details['descriptions'])
                    additional_info = " | ".join(word_details['additionalInfos'])
                    update_date = word_details['updateDate']

                    # CSV 파일에 저장
                    writer.writerow([word_text, part_of_speech, dark_text, description, additional_info, update_date])
                    print(f'✅ Successfully saved: {word_text}')

                    # 상세 페이지에서 뒤로가기
                    driver.back()
                    time.sleep(2)

                except Exception as e:
                    print(f'❌ Failed to process word: {word_text} - {str(e)}')
                    continue
            
            # 다음 페이지로 이동
            try:
                next_button = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.arrows__next'))
                )
                driver.execute_script("arguments[0].click();", next_button)
                print(f'➡️ Moving to page {page + 1}')
                time.sleep(2)

            except Exception as e:
                print(f'❌ 다음 페이지로 이동 실패: {str(e)}')
                break

        except Exception as e:
            print(f"❌ 페이지 {page} 처리 실패: {str(e)}")
            continue

# 드라이버 종료
driver.quit()
print('🎉 Crawling completed and saved to', OUTPUT_CSV)
